In [1]:
import pandas as pd
import ast
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore

In [2]:
file = "../Files/preprocessed_5percent_7_3.csv"
data = pd.read_csv("../Files/preprocessed_5percent_7_3.csv", converters={"item1a_risk":ast.literal_eval,
                                                                  "item7_mda":ast.literal_eval})

In [3]:
# Forgot to lowercase before bcuz im a dumbass lol
for label in ['item1a_risk', 'item7_mda']:
    data[label] = data[label].map(lambda doc: [list(map(str.lower, sentence_grp)) for sentence_grp in doc])

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,cik,ticker_x,filing_date,item1a_risk,item7_mda,year_x,filing_year_x,...,ticker_y,year_y,company_name,is_dividend_payer,dps_change,is_dps_cut,z_environmental,d_environmental,sector,filing_year_y
0,0,262,262,775158,OSK,2010-11-18,"[[item, risk, factors, the, company, financial...","[[item, management, discussion, and, analysis,...",2010,2010,...,OSK,2010,OSHKOSH CORP,0.0,NaN,NaN,NaN,NaN,Industrials,2010
1,1,9070,9070,780571,ITRI,2014-02-26,"[[item, risk, factorswe, dependent, utility, i...","[[item, managements, discussion, and, analysis...",2014,2014,...,ITRI,2013,ITRON INC,0.0,NaN,NaN,1.450191,0.271210,Tech,2014
2,2,18025,18025,1169561,CVLT,2017-05-05,"[[item, risk, factorsyou, consider, following,...","[[item, management, discussion, analysis, fina...",2017,2017,...,CVLT,2017,COMMVAULT SYSTEMS INC,0.0,NaN,NaN,-0.783233,-0.100064,Tech,2017
3,3,7453,7453,1093649,IDIX,2013-02-25,"[[item, risk, factors, our, business, face, ri...","[[item, management, discussion, analysis, fina...",2013,2013,...,IDIX,2012,IDENIX PHARMACEUTICALS INC,0.0,NaN,NaN,-1.180099,3.860642,Health Care,2013
4,4,5361,5361,1332602,CBOU,2012-03-16,"[[item, risk, factors, certain, statement, fil...","[[item, management, discussion, analysis, fina...",2012,2012,...,CBOU,2012,CARIBOU COFFEE CO,0.0,NaN,NaN,NaN,NaN,Consumer,2012


In [5]:
params = {
    'num_topics': 30,
    'chunksize': 2000,
    'passes': 20,
    'iterations': 400,
    'eval_every': None,
    'alpha': 'asymmetric',
    'eta': 'auto'
}

In [27]:
data["item1a_risk"][0][0]

['item',
 'risk',
 'factors',
 'the',
 'company',
 'financial',
 'position',
 'result',
 'operation',
 'cash',
 'flow',
 'subject',
 'risk',
 'exclusively',
 'company',
 'control',
 'cause',
 'actual',
 'performance',
 'differ',
 'materially',
 'historical',
 'projected',
 'future',
 'performance',
 'investors',
 'consider',
 'carefully',
 'information',
 'form',
 '10k',
 'light',
 'risk',
 'factor',
 'described',
 'information',
 'set',
 'forth',
 'caption',
 'management',
 'discussion',
 'analysis',
 'financial',
 'condition',
 'results',
 'operations',
 'certain',
 'assumptions',
 'certain',
 'market',
 'highly',
 'cyclical',
 'current',
 'decline',
 'market',
 'material',
 'adverse',
 'effect',
 'operating',
 'performance',
 'the',
 'high',
 'level',
 'sale',
 'defense',
 'business',
 'recent',
 'year',
 'significant',
 'demand',
 'defense',
 'truck',
 'replacement',
 'part',
 'service',
 'including',
 'armoring',
 'truck',
 'remanufacturing',
 'arising',
 'conflict',
 'iraq',
 'af

In [6]:
original_slice = data[(data.year_x >= 2009) & (data.year_x <= 2011)]

# for label in ['item1a_risk', 'item7_mda']:
documents = [sentence_grp for doc in original_slice["item1a_risk"].to_list() for sentence_grp in doc]

dictionary = Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]
_temp = dictionary[0] # Initialize id2token mappings
id2word = dictionary.id2token

In [7]:
lda = LdaMulticore(
        corpus=corpus,
        id2word=id2word,
        workers=8,
        **params
    )

In [14]:
import copy

In [15]:
orig = copy.deepcopy(lda)

In [11]:
new_slice = data[data.year_x == 2012]

In [12]:
display(len(new_slice))

122

In [ ]:
orig.

In [22]:
orig.show_topics()[-1]

(0,
 '0.035*"loan" + 0.023*"investment" + 0.014*"mortgage" + 0.014*"loss" + 0.014*"risk" + 0.014*"rate" + 0.013*"security" + 0.013*"asset" + 0.012*"market" + 0.011*"property"')

In [23]:
lda.show_topics()[-1]

(0,
 '0.031*"loan" + 0.028*"investment" + 0.024*"rate" + 0.023*"property" + 0.015*"risk" + 0.014*"loss" + 0.014*"real" + 0.013*"security" + 0.013*"asset" + 0.012*"estate"')

In [13]:
new_documents = [sentence_grp for doc in new_slice["item1a_risk"].to_list() for sentence_grp in doc]

new_dictionary = Dictionary(new_documents)
corpus = [dictionary.doc2bow(doc) for doc in new_documents]
_temp = new_dictionary[0] # Initialize id2token mappings
id2word = new_dictionary.id2token

In [16]:
lda.update(corpus)

In [16]:
a = data["item1a_risk"].to_list()

In [22]:
#b = [group for group in doc for doc in data["item1a_risk"].to_list()]
b = [sentence_grp for doc in data["item1a_risk"].to_list() for sentence_grp in doc]

In [ ]:
start = 2009
end = 2012
for label in ['item1a_risk', 'item7_mda']:
    

In [35]:
documents = [sentence_grp for doc in data["item1a_risk"].to_list() for sentence_grp in doc]

In [37]:
dictionary = Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [38]:
_temp = dictionary[0] # Initialize id2token mappings
id2word = dictionary.id2token

In [ ]:
lda = LdaMulticore()